In [1]:
from time import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from ImagesloaderRAM import *
from Trainfunction import *
from Savemodel import *
from Mycudatransformation import *
from MynnModule import *
from PSNR import *

import torchvision.transforms as T
import torchvision.utils as tu

import numpy as np

%load_ext autoreload
%autoreload 2


dtype = torch.FloatTensor
gpu_dtype = torch.cuda.FloatTensor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


print('using device:', device)

using device: cuda


## Train

In [5]:
# Loading the dataset of images


Data_preprocessing = T.Compose([T.RandomCrop(64),
                                T.ToTensor()])

trainset = ImageFolderRAM('./BSD200C', 0.1, Data_preprocessing,
                          Randomnoise=True, loader=RGB_loader, loadram='cpu')

trainloader = torch.utils.data.DataLoader(
    trainset, shuffle=True, batch_size=64, num_workers=1)

## Training from scratch

In [6]:
# Choose the model of neural networks architectures
Denoiser = SFD_C().to(device)

# Choose the loss function
loss = nn.L1Loss().to(device)

# Choose the Optimizer
optimizer = optim.Adam(Denoiser.parameters(), lr=0.001, betas=(0.9, 0.999),
                       eps=1e-08, weight_decay=0.00001, amsgrad=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.9, patience=5000, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

In [7]:
def trainserver(model, loader_train, loss_fn, optimizer, scheduler, num_epochs=1, save_every=1, loss_every=10, filename='denoiser'):

    loss_history = []
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()

        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.to(device))
            y_var = Variable(y.to(device))

            frame = model(x_var)
            loss = loss_fn(frame, y_var)

            if (t + 1) % loss_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data))
                loss_history.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % save_every == 0:
            Save_modelloss(model, optimizer, loss_history,
                           filename+'%s ' % int(epoch+1))

        scheduler.step(loss.data)

In [8]:
trainserver(Denoiser, trainloader, loss, optimizer, scheduler,
            num_epochs=500000, save_every=5000, loss_every=3, filename='SFD_C_25NB')

Starting epoch 1 / 500000
t = 3, loss = 0.2973
Starting epoch 2 / 500000
t = 3, loss = 0.1335
Starting epoch 3 / 500000
t = 3, loss = 0.0988
Starting epoch 4 / 500000
t = 3, loss = 0.1116
Starting epoch 5 / 500000
t = 3, loss = 0.0994
Starting epoch 6 / 500000
t = 3, loss = 0.0884
Starting epoch 7 / 500000
t = 3, loss = 0.1005
Starting epoch 8 / 500000
t = 3, loss = 0.0755
Starting epoch 9 / 500000
t = 3, loss = 0.0790
Starting epoch 10 / 500000
t = 3, loss = 0.0890
Starting epoch 11 / 500000
t = 3, loss = 0.0812
Starting epoch 12 / 500000
t = 3, loss = 0.0761
Starting epoch 13 / 500000
t = 3, loss = 0.0664
Starting epoch 14 / 500000
t = 3, loss = 0.0786
Starting epoch 15 / 500000
t = 3, loss = 0.0781
Starting epoch 16 / 500000
t = 3, loss = 0.0618
Starting epoch 17 / 500000
t = 3, loss = 0.0599
Starting epoch 18 / 500000
t = 3, loss = 0.0689
Starting epoch 19 / 500000
t = 3, loss = 0.0639
Starting epoch 20 / 500000
t = 3, loss = 0.0623
Starting epoch 21 / 500000
t = 3, loss = 0.0529
S

Starting epoch 170 / 500000
t = 3, loss = 0.0453
Starting epoch 171 / 500000
t = 3, loss = 0.0491
Starting epoch 172 / 500000
t = 3, loss = 0.0371
Starting epoch 173 / 500000
t = 3, loss = 0.0410
Starting epoch 174 / 500000
t = 3, loss = 0.0458
Starting epoch 175 / 500000
t = 3, loss = 0.0361
Starting epoch 176 / 500000
t = 3, loss = 0.0288
Starting epoch 177 / 500000
t = 3, loss = 0.0346
Starting epoch 178 / 500000
t = 3, loss = 0.0431
Starting epoch 179 / 500000
t = 3, loss = 0.0340
Starting epoch 180 / 500000
t = 3, loss = 0.0281
Starting epoch 181 / 500000
t = 3, loss = 0.0544
Starting epoch 182 / 500000
t = 3, loss = 0.0532
Starting epoch 183 / 500000
t = 3, loss = 0.0472
Starting epoch 184 / 500000
t = 3, loss = 0.0361
Starting epoch 185 / 500000
t = 3, loss = 0.0355
Starting epoch 186 / 500000
t = 3, loss = 0.0737
Starting epoch 187 / 500000
t = 3, loss = 0.0438
Starting epoch 188 / 500000
t = 3, loss = 0.0477
Starting epoch 189 / 500000
t = 3, loss = 0.0499
Starting epoch 190 /

Starting epoch 337 / 500000
t = 3, loss = 0.0447
Starting epoch 338 / 500000
t = 3, loss = 0.0616
Starting epoch 339 / 500000
t = 3, loss = 0.0383
Starting epoch 340 / 500000
t = 3, loss = 0.0335
Starting epoch 341 / 500000
t = 3, loss = 0.0379
Starting epoch 342 / 500000
t = 3, loss = 0.0409
Starting epoch 343 / 500000
t = 3, loss = 0.0558
Starting epoch 344 / 500000
t = 3, loss = 0.0480
Starting epoch 345 / 500000
t = 3, loss = 0.0617
Starting epoch 346 / 500000
t = 3, loss = 0.0458
Starting epoch 347 / 500000
t = 3, loss = 0.0373
Starting epoch 348 / 500000
t = 3, loss = 0.0390
Starting epoch 349 / 500000
t = 3, loss = 0.0318
Starting epoch 350 / 500000
t = 3, loss = 0.0343
Starting epoch 351 / 500000
t = 3, loss = 0.0330
Starting epoch 352 / 500000
t = 3, loss = 0.0308
Starting epoch 353 / 500000
t = 3, loss = 0.0445
Starting epoch 354 / 500000
t = 3, loss = 0.0426
Starting epoch 355 / 500000
t = 3, loss = 0.0384
Starting epoch 356 / 500000
t = 3, loss = 0.0437
Starting epoch 357 /

Starting epoch 504 / 500000
t = 3, loss = 0.0323
Starting epoch 505 / 500000
t = 3, loss = 0.0330
Starting epoch 506 / 500000
t = 3, loss = 0.0316
Starting epoch 507 / 500000
t = 3, loss = 0.0260
Starting epoch 508 / 500000
t = 3, loss = 0.0338
Starting epoch 509 / 500000
t = 3, loss = 0.0250
Starting epoch 510 / 500000
t = 3, loss = 0.0394
Starting epoch 511 / 500000
t = 3, loss = 0.0409
Starting epoch 512 / 500000
t = 3, loss = 0.0337
Starting epoch 513 / 500000
t = 3, loss = 0.0433
Starting epoch 514 / 500000
t = 3, loss = 0.0359
Starting epoch 515 / 500000
t = 3, loss = 0.0421
Starting epoch 516 / 500000
t = 3, loss = 0.0430
Starting epoch 517 / 500000
t = 3, loss = 0.0453
Starting epoch 518 / 500000
t = 3, loss = 0.0298
Starting epoch 519 / 500000
t = 3, loss = 0.0647
Starting epoch 520 / 500000
t = 3, loss = 0.0522
Starting epoch 521 / 500000
t = 3, loss = 0.0251
Starting epoch 522 / 500000
t = 3, loss = 0.0345
Starting epoch 523 / 500000
t = 3, loss = 0.0342
Starting epoch 524 /

Starting epoch 671 / 500000
t = 3, loss = 0.0313
Starting epoch 672 / 500000
t = 3, loss = 0.0279
Starting epoch 673 / 500000
t = 3, loss = 0.0283
Starting epoch 674 / 500000
t = 3, loss = 0.0314
Starting epoch 675 / 500000
t = 3, loss = 0.0322
Starting epoch 676 / 500000
t = 3, loss = 0.0322
Starting epoch 677 / 500000
t = 3, loss = 0.0624
Starting epoch 678 / 500000
t = 3, loss = 0.0264
Starting epoch 679 / 500000
t = 3, loss = 0.0293
Starting epoch 680 / 500000
t = 3, loss = 0.0661
Starting epoch 681 / 500000
t = 3, loss = 0.0353
Starting epoch 682 / 500000
t = 3, loss = 0.0385
Starting epoch 683 / 500000
t = 3, loss = 0.0520
Starting epoch 684 / 500000
t = 3, loss = 0.0319
Starting epoch 685 / 500000
t = 3, loss = 0.0415
Starting epoch 686 / 500000
t = 3, loss = 0.0305
Starting epoch 687 / 500000
t = 3, loss = 0.0309
Starting epoch 688 / 500000
t = 3, loss = 0.0278
Starting epoch 689 / 500000
t = 3, loss = 0.0476
Starting epoch 690 / 500000
t = 3, loss = 0.0340
Starting epoch 691 /

...

Starting epoch 9507 / 500000
t = 3, loss = 0.0194
Starting epoch 9508 / 500000
t = 3, loss = 0.0224
Starting epoch 9509 / 500000
t = 3, loss = 0.0196
Starting epoch 9510 / 500000
t = 3, loss = 0.0196
Starting epoch 9511 / 500000
t = 3, loss = 0.0201
Starting epoch 9512 / 500000
t = 3, loss = 0.0260
Starting epoch 9513 / 500000
t = 3, loss = 0.0234
Starting epoch 9514 / 500000
t = 3, loss = 0.0211
Starting epoch 9515 / 500000
t = 3, loss = 0.0232
Starting epoch 9516 / 500000
t = 3, loss = 0.0249
Starting epoch 9517 / 500000
t = 3, loss = 0.0207
Starting epoch 9518 / 500000
t = 3, loss = 0.0256
Starting epoch 9519 / 500000
t = 3, loss = 0.0220
Starting epoch 9520 / 500000
t = 3, loss = 0.0200
Starting epoch 9521 / 500000
t = 3, loss = 0.0194
Starting epoch 9522 / 500000
t = 3, loss = 0.0235
Starting epoch 9523 / 500000
t = 3, loss = 0.0211
Starting epoch 9524 / 500000
t = 3, loss = 0.0270
Starting epoch 9525 / 500000
t = 3, loss = 0.0191
Starting epoch 9526 / 500000
t = 3, loss = 0.0207


t = 3, loss = 0.0200
Starting epoch 9671 / 500000
t = 3, loss = 0.0240
Starting epoch 9672 / 500000
t = 3, loss = 0.0207
Starting epoch 9673 / 500000
t = 3, loss = 0.0238
Starting epoch 9674 / 500000
t = 3, loss = 0.0228
Starting epoch 9675 / 500000
t = 3, loss = 0.0179
Starting epoch 9676 / 500000
t = 3, loss = 0.0273
Starting epoch 9677 / 500000
t = 3, loss = 0.0212
Starting epoch 9678 / 500000
t = 3, loss = 0.0206
Starting epoch 9679 / 500000
t = 3, loss = 0.0194
Starting epoch 9680 / 500000
t = 3, loss = 0.0194
Starting epoch 9681 / 500000
t = 3, loss = 0.0224
Starting epoch 9682 / 500000
t = 3, loss = 0.0218
Starting epoch 9683 / 500000
t = 3, loss = 0.0190
Starting epoch 9684 / 500000
t = 3, loss = 0.0218
Starting epoch 9685 / 500000
t = 3, loss = 0.0238
Starting epoch 9686 / 500000
t = 3, loss = 0.0215
Starting epoch 9687 / 500000
t = 3, loss = 0.0217
Starting epoch 9688 / 500000
t = 3, loss = 0.0206
Starting epoch 9689 / 500000
t = 3, loss = 0.0308
Starting epoch 9690 / 500000


Starting epoch 9834 / 500000
t = 3, loss = 0.0211
Starting epoch 9835 / 500000
t = 3, loss = 0.0241
Starting epoch 9836 / 500000
t = 3, loss = 0.0185
Starting epoch 9837 / 500000
t = 3, loss = 0.0196
Starting epoch 9838 / 500000
t = 3, loss = 0.0194
Starting epoch 9839 / 500000
t = 3, loss = 0.0195
Starting epoch 9840 / 500000
t = 3, loss = 0.0201
Starting epoch 9841 / 500000
t = 3, loss = 0.0203
Starting epoch 9842 / 500000
t = 3, loss = 0.0183
Starting epoch 9843 / 500000
t = 3, loss = 0.0217
Starting epoch 9844 / 500000
t = 3, loss = 0.0179
Starting epoch 9845 / 500000
t = 3, loss = 0.0218
Starting epoch 9846 / 500000
t = 3, loss = 0.0227
Starting epoch 9847 / 500000
t = 3, loss = 0.0219
Starting epoch 9848 / 500000
t = 3, loss = 0.0186
Starting epoch 9849 / 500000
t = 3, loss = 0.0227
Starting epoch 9850 / 500000
t = 3, loss = 0.0225
Starting epoch 9851 / 500000
t = 3, loss = 0.0188
Starting epoch 9852 / 500000
t = 3, loss = 0.0223
Starting epoch 9853 / 500000
t = 3, loss = 0.0238


t = 3, loss = 0.0290
Starting epoch 9998 / 500000
t = 3, loss = 0.0219
Starting epoch 9999 / 500000
t = 3, loss = 0.0224
Starting epoch 10000 / 500000
t = 3, loss = 0.0291
Starting epoch 10001 / 500000
t = 3, loss = 0.0210
Starting epoch 10002 / 500000
t = 3, loss = 0.0262
Starting epoch 10003 / 500000
t = 3, loss = 0.0232
Starting epoch 10004 / 500000
t = 3, loss = 0.0216
Starting epoch 10005 / 500000
t = 3, loss = 0.0200
Starting epoch 10006 / 500000
t = 3, loss = 0.0197
Starting epoch 10007 / 500000
t = 3, loss = 0.0204
Starting epoch 10008 / 500000
t = 3, loss = 0.0202
Starting epoch 10009 / 500000
t = 3, loss = 0.0283
Starting epoch 10010 / 500000
t = 3, loss = 0.0216
Starting epoch 10011 / 500000
t = 3, loss = 0.0213
Starting epoch 10012 / 500000
t = 3, loss = 0.0221
Starting epoch 10013 / 500000
t = 3, loss = 0.0184
Starting epoch 10014 / 500000
t = 3, loss = 0.0226
Starting epoch 10015 / 500000
t = 3, loss = 0.0211
Starting epoch 10016 / 500000
t = 3, loss = 0.0199
Starting epo

KeyboardInterrupt: 

### Load from a previous training

In [ ]:
filename = 'SFD_B75000'

In [ ]:
Denoiser, optimizer, loss = Load_modelloss(filename)
loss = nn.L1Loss().to(device)

In [ ]:
def trainserver(model, loader_train, loss_fn, optimizer, scheduler, num_epochs=1, save_every=1, loss_every=10, filename='denoiser'):

    loss_history = []
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()

        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.to(device))
            y_var = Variable(y.to(device))

            frame = model(x_var)
            loss = loss_fn(frame, y_var)

            if (t + 1) % loss_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data))
                loss_history.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % save_every == 0:
            Save_modelloss(model, optimizer, loss_history,
                           filename+'%s ' % int(epoch+1))

        scheduler.step(loss.data)

In [ ]:
trainserver(Denoiser, trainloader, loss, optimizer, scheduler,
            num_epochs=500000, save_every=25000, loss_every=3, filename=filename)

# Test denoiser and save images

In [ ]:
Data_test = T.Compose([
    T.ToTensor()])

testset = ImageFolderRAM('./CBSD68', 0.1, Data_test,
                         Randomnoise=False, loadram=False, loader=RGB_loader)

testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size=1)

In [ ]:
filename1 = 'SFD_25SFD_C_NB75000'
filename1 = 'SFD_25SFD_C_NB75000'
Denoiser, optimizer, loss = Load_modelloss(filename1)
Denoiser2, optimizer, loss2 = Load_modelloss(filename2)



In [ ]:
def Show_result(Denoiser, loader, pause, check=False):
    """
Fonction permettant de montrer les resultats de debuitages d'un reseau de neuronnes
Input :
    - model : reseau à tester
    - loader_train : l'ensemble d'image issu du dataloader
    - pause : temps de pause entre chaque image
    - check : if True cela affiche la moyenne de l'image, le min , le max

Affiche :
    -L'image normale
    -L'image bruitée
    -L'image debruité
    -Les PSNR
"""
    Denoiser.eval()

    plt.ion()
    plt.show()

    # Calcul de la PSNR moyenne
    PSNRmoy = 0
    Compteur = 0

    for i, data in enumerate(loader):
        images, groundtrue = data

        plt.figure(1)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            groundtrue, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        print("Image bruitée")
        plt.figure(2)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        plt.title(r'%f ' % A)

        print("Image débruitée")
        plt.figure(3)
        plt.clf()
        images = Denoiser(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        PSNRmoy += A
        Compteur += 1
        plt.title(r'%f ' % A)

        plt.pause(pause)

    print('PSNR moyen = %f ' % (PSNRmoy/Compteur))

In [ ]:
def Show_result2(Denoiser1, Denoiser2, loader, pause, check=False):
    """
Fonction permettant de comparer les resultats de debuitages d'un reseau de neuronnes
Input :
    - Denoiser1 : debruiteur 1 à tester
    - Denoiser2 : debruiteur 2 à tester
    - loader_train : l'ensemble d'image issu du dataloader
    - pause : temps de pause entre chaque image
    - check : if True cela affiche la moyenne de l'image, le min , le max

Affiche :
    - L'image normale
    - L'image bruitée
    - L'image debruité par 1
    - L'image debruité par 2
    - Les PSNR
"""
    Denoiser1.eval()
    Denoiser2.eval()

    plt.ion()
    plt.show()

    # Calcul de la PSNR moyenne
    PSNRmoy1 = 0
    PSNRmoy2 = 0
    Compteur = 0

    for i, data in enumerate(loader):
        images, groundtrue = data

        plt.figure(1)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            groundtrue, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        print("Image bruitée")
        plt.figure(2)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        plt.title(r'%f ' % A)

        print("Image débruitée par 1")
        plt.figure(3)
        plt.clf()
        images1 = Denoiser1(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images1, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images1.numpy(), check=check)
        PSNRmoy1 += A
        Compteur += 1
        plt.title(r' Denoiseur1 PSNR=%f ' % A)

        print("Image débruitée par 2")
        plt.figure(4)
        plt.clf()
        images2 = Denoiser2(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images2, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images2.numpy())
        PSNRmoy2 += A
        plt.title(r' Denoiseur2 PSNR= %f ' % A)

        plt.pause(pause)

    print(' Mean PSNR denoiseur1 = %f ' % (PSNRmoy1/Compteur))
    print(' Mean PSNR denoiseur2 = %f ' % (PSNRmoy2/Compteur))

In [ ]:
# For only one denoiser
Show_result(Denoiser, testloader, 0.1, check=False)

In [ ]:
# To compare 2de
Show_result2(Denoiser, Denoiser2, testloader, 0.1, check=False)

In [ ]:
# Loss Plot

##
plt.show()
plt.ion()

##
plt.figure(5)
plt.clf()
plt.semilogy(range(0, len(loss2))[:10000:10], loss2[:10000:10])

plt.figure(6)
plt.clf()
plt.plot(range(0, len(loss))[:10000:10], loss[:10000:10])

##
plt.figure(6)
plt.clf()
plt.plot(range(0, len(loss2))[1::10], loss2[10000::10])


##
plt.figure(7)
plt.clf()
plt.plot(range(0, len(loss3)), loss3)